In [1]:
# ! pip install btrdb

import btrdb
import pandas as pd
import os
import uuid

from datetime import datetime, timedelta

%timeit

In [4]:
%%timeit
api_key = '' #Insert your own API key 

db = btrdb.connect("api.ni4ai.org:4411", apikey=api_key)

202 µs ± 16.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [10]:
t = datetime.now()
path = os.path.join('..','..','data','disturbance')
df = pd.read_csv(os.path.join(path, 'texas_data.csv'))

print(datetime.now()-t)

0:00:00.160310


In [11]:
df.head()

,Timestamp,CLOUD:V1LPM_Magnitude,CLOUD:V1LPM_Angle,HARRIS:V1LPM_Magnitude,HARRIS:V1LPM_Angle,CONROE:V1LPM_Magnitude,CONROE:V1LPM_Angle,CLOUD:Frequency,HARRIS:Frequency,CONROE:Frequency
0,00:00.0,40.3226,32.5216,67209.5781,-173.3824,40097.7813,135.5679,60.0218,59.9947,59.9657
1,00:00.0,40.3318,32.8078,67206.1719,-173.4610,40099.1250,135.1568,60.0227,59.9944,59.9656
2,00:00.1,40.3372,33.0900,67201.6719,-173.5453,40099.2148,134.7479,60.0233,59.9940,59.9656
3,00:00.1,40.3345,33.3569,67199.3125,-173.6182,40093.8633,134.3295,60.0233,59.9938,59.9656
4,00:00.1,40.3272,33.6064,67196.1797,-173.6889,40085.8984,133.8997,60.0231,59.9938,59.9654


In [12]:
nomenclature = {'CLOUD:V1LPM_Magnitude': ('whitesands/voltage_magnitude', 'volts'),
                'CLOUD:V1LPM_Angle': ('whitesands/voltage_angle', 'deg'),
                
                'HARRIS:V1LPM_Magnitude': ('centex/voltage_magnitude', 'volts'),
                'HARRIS:V1LPM_Angle': ('centex/voltage_angle', 'deg'),
                
                'CONROE:V1LPM_Magnitude': ('conroe/voltage_magnitude', 'volts'),
                'CONROE:V1LPM_Angle': ('conroe/voltage_angle', 'deg'),
                
                'CLOUD:Frequency': ('whitesands/frequency','Hz'),
                'HARRIS:Frequency': ('centex/frequency','Hz'),
                'CONROE:Frequency': ('conroe/frequency','Hz'),
               }

In [13]:
datetimes = pd.date_range(start=datetime(2020,9,7,0,0), 
                          end=datetime(2020,9,7,1,0), 
                          freq='%ins'%(1e9*1./30))

ns_timestamps = [btrdb.utils.timez.to_nanoseconds(t) for t in datetimes]
ns_timestamps = ns_timestamps[:len(df)]

In [17]:
t = datetime.now()
dataset = "lndunn/brownout"

points = 0
streams = 0
for col in df.keys():
    if col == 'Timestamp':
        pass
    else:
        stream_name, units = nomenclature[col]
        sensor_name, stream_name = os.path.split(stream_name)
        collection = os.path.join(dataset, sensor_name)
        
        stream = db.create(uuid=uuid.uuid4(),
                        collection=collection,
                        tags={"name": stream_name, "unit": units})
            
        # insert values into the new stream
        stream.insert(list(zip(ns_timestamps, df[col].tolist())))
        
        print(stream_name, len(df))
        points += len(df)
        streams += 1
        
print(points, streams)

print(datetime.now()-t)

voltage_magnitude 107959
voltage_angle 107959
voltage_magnitude 107959
voltage_angle 107959
voltage_magnitude 107959
voltage_angle 107959
frequency 107959
frequency 107959
frequency 107959
971631 9
0:00:11.640653


In [16]:
def remove_streams(collection, stream_names=None):
    if stream_names == None:
        streams = db.streams_in_collection(collection)
        for s in streams:
            s.obliterate()
            print(s)
            
remove_streams('lndunn/brownout')

<Stream collection=lndunn/brownout/cloud name=voltage_magnitude>
<Stream collection=lndunn/brownout/cloud name=voltage_angle>
<Stream collection=lndunn/brownout/harris name=voltage_magnitude>
<Stream collection=lndunn/brownout/harris name=voltage_angle>
<Stream collection=lndunn/brownout/conroe name=voltage_magnitude>
<Stream collection=lndunn/brownout/conroe name=voltage_angle>
<Stream collection=lndunn/brownout/cloud name=frequency>
<Stream collection=lndunn/brownout/harris name=frequency>
<Stream collection=lndunn/brownout/conroe name=frequency>
<Stream collection=lndunn/brownout/whitesands name=voltage_magnitude>
<Stream collection=lndunn/brownout/whitesands name=voltage_angle>
<Stream collection=lndunn/brownout/centex name=voltage_magnitude>
<Stream collection=lndunn/brownout/centex name=voltage_angle>


78


In [36]:
points = 0
for s in streams:
    print(ns_to_datetime(s.latest()[0][0]))
    windows, _ = zip(*s.aligned_windows(datetime(2001,1,1), datetime.now(), 62))
    for window in windows:
        print(window.count)
        points += window.count

NameError: name 'ns_to_datetime' is not defined